# Week 9 Lab: Lists, Dictionaries, and Panda
This week’s lab gives you practical experience with data analysis in Python.

You will:
- Traverse lists using for loops and the accumulator pattern
- Use dictionaries to represent structured data and practice common iteration patterns
- Load, access, and explore data using pandas DataFrames

**Instructions**
- Work through the problems in order.
- Write tests where indicated and run them to verify your progress.


#### Run the cell below once to set up the test environment.

In [ ]:
import piplite
await piplite.install(["pytest", "ipytest"])

import ipytest
ipytest.autoconfig()

## Problem 1: Movie Ratings Dashboard 
**Focus:** Lists, loops, accumulator pattern, dictionaries

You are designing a simple analytics utility for a movie review site.

### Task 1.1 – Summing and Averaging Ratings
Implement `average_rating(ratings)` to compute the mean rating (return `0.0` for empty lists). Use a **loop + accumulator**. (avoid using sum()/len() directly for practice). 

**Write some test cases.**

In [ ]:
# Implement using a loop + accumulator
def average_rating(ratings: list[int | float]) -> float:
    """Return the average rating for a movie. Return 0.0 if list is empty.
    Use a loop+accumulator (avoid using sum()/len() directly for practice).
    """
    # TODO: Replace with your implementation (accumulator pattern)
    if not ratings:
        return 0.0
    total = 0.0
    count = 0
    for r in ratings:
        total += r
        count += 1
    return total / count if count > 0 else 0.0


In [ ]:
%%ipytest -qq
# Your test cases here
def test_average_rating():
    assert abs(average_rating([4, 5, 3, 5]) - 4.25) < 1e-9
    assert average_rating([]) == 0.0

### Use the Below Movie Ratings Dictionary for the Next Two Tasks

In [ ]:
movie_ratings = {
    "Inception": [5, 4, 5, 5, 4],
    "Avatar": [4, 3, 4, 4],
    "Titanic": [5, 5, 4, 5],
    "Joker": [3, 3.5, 4]
}
movie_ratings

### Task 1.2 – Compute Average Ratings per Movie
Implement `print_movie_averages(movies)` that iterates and prints each movie with its average rating using your function from Task 1.1.

In [ ]:
def print_movie_averages(movies: dict[str, list[int | float]]) -> None:
    """Print each movie and its average rating using average_rating()."""
    # TODO: Implement using a loop
    for title, ratings in movies.items():
        avg = average_rating(ratings)
        print(f"{title}: {avg:.2f}")

print_movie_averages(movie_ratings)


### Task 1.3 – Reverse Engineering a Function
Implement a function `filter_by_threshold(movies, threshold)` so that all tests in the next cell pass.

In [ ]:
def filter_by_threshold(movies: dict[str, list[float]], threshold: float) -> list[str]:
    """Return titles whose average rating is strictly greater than threshold."""
    # TODO: Implement using a loop
    result: list[str] = []
    for title, ratings in movies.items():
        if average_rating(ratings) > threshold:
            result.append(title)
    return result

In [ ]:
%%ipytest -qq

def test_filter_by_threshold():
    assert filter_by_threshold(movie_ratings, 4.0) == ['Inception', 'Titanic']
    assert filter_by_threshold(movie_ratings, 4.6) == ['Titanic']
    assert filter_by_threshold(movie_ratings, 3.0) == ['Inception', 'Avatar', 'Titanic', 'Joker']


---
## Problem 2: Pandas (Rows, Columns, Basic Analysis)

You will practice **exactly** the core operations from the lecture:
- `pd.read_csv`
- `head()` / `tail()`
- Row access with `iloc` (including slicing)
- Column access with `orders['column']`
- Series operations: `.mean()`, `.sum()`, `.unique()`

Dataset: **`retail_orders.csv`** (coffee shop sales)

**Columns:** `order_id, date, branch, item, size, quantity, unit_price, order_type, payment_method`


### Load the data

Use `pd.read_csv` and preview the first few rows.

In [ ]:
import pandas as pd
orders = pd.read_csv('retail_orders.csv')
orders.head()

### Task 2.1 - Row and Column access
1. Show rows **5 to 9** (remember slicing excludes the end index).
2. Show every **10th** row starting at 0.
3. Show the **last row** using negative indexing with `iloc`.
4. Get the `quantity` column as a Series and show the first 8 values.
5. Get the **unique** values of `order_type`.

In [ ]:
# TODO: 1 Rows 5 to 9
orders.iloc[5:10]

In [ ]:
# TODO: 2 Every 10th row
orders.iloc[::10]

In [ ]:
# TODO: 3 Last row
orders.iloc[-1]

In [ ]:
# TODO: 4 Quantity first 8
orders['quantity'].head(8)

In [ ]:
# TODO: 5 Unique order_type
orders['order_type'].unique()

### Task 2.2 - Create a derived column
Create `total_price = quantity * unit_price` using simple arithmetic. Then preview with `head()`.

In [ ]:
# TODO: Create total_price column then preview
orders['total_price'] = orders['quantity'] * orders['unit_price']
orders.head()

### Task 2.3 - Basic analyses using column selection and Series methods
Use `Boolean` filters inside []. Also use Series methods: `.sum()`/`.mean()`/`.unique()`:<br>
- **Total quantity** sold for the item `'Latte'` (e.g., `orders[orders['item'] == 'Latte']['quantity'].sum()`).<br>
    - `orders['item'] == 'Latte'`: This expression creates a Boolean Series (a list of True/False values), one per row.
    - `orders[orders['item'] == 'Latte']`: This uses Boolean indexing to filter the DataFrame, keeping only the rows where the condition is True.
    - `['quantity']`: From that filtered DataFrame, you now select just the 'quantity' column.
- **Average quantity** for orders with `order_type == 'takeout'`.<br>
- **Unique** items sold at the `'University'` branch.

**Windowed comparisons:** 
<br>
- For rows **0 – 24**, compute the **mean quantity**.<br>
- For rows **25 – 49**, compute the **mean quantity**.<br>
- Which window has the higher mean? 

In [ ]:
# TODO: Total quantity for Latte 
orders[orders['item'] == 'Latte']['quantity'].sum()

In [ ]:
# TODO: Mean quantity for takeout
orders[orders['order_type']=='takeout']['quantity'].mean()

In [ ]:
# TODO: Unique items at University
orders[orders['branch']=='University']['item'].unique()

In [ ]:
# TODO: Compute window means and compare
mean_a = orders.iloc[0:25]['quantity'].mean()
mean_b = orders.iloc[25:50]['quantity'].mean()
mean_a, mean_b

### Task 2.4 - Total Quantity
Compute the **total quantity** for each of these items:
`['Latte', 'Espresso', 'Cappuccino']` by summing filtered Series.
Then figure out which of the three has the highest total.

**Instructions:**
- Write a function that: 
    - Takes the given DataFrame (`item_df`) containing an 'item' column.
    - Loops through each item name in that column.
    - Filters the main orders DataFrame to select only the rows where the 'item' matches.
    - Stores each result in a dictionary where the key is the item name and the value is its total quantity.
        - (*Hint*) Uses ['quantity'].sum() to compute the total quantity
    - Returns that dictionary.

In [ ]:
# TODO: Write a function to compute totals and find the max
items = {'item': ['Latte','Espresso','Cappuccino']}
item_df = pd.DataFrame(items)
def compute_totals(item_DataFrame: pd.DataFrame) -> dict[str, int]:
    totals = {}
    for it in item_DataFrame['item']:
        totals[it] = orders[orders['item']==it]['quantity'].sum()
    return totals
compute_totals(item_df)
